# Practico 4

### I) Implementar una solución en Python que permita realizar un análisis exploratorio del conjunto de datos “movies.csv” de Kaggle, que contiene datos descargados usando BeautifulSoup de IMBD

#### a) Descargar los datos desde https://ignaciorlando.github.io/datasets/data-science/movies.csv.

In [56]:
from wget import download                   # type: ignore
from os import path                         # type: ignore
import pandas as pd                         # type: ignore


if not path.exists("movies.csv") :
    download("https://ignaciorlando.github.io/datasets/data-science/movies.csv")
else :
    print("El archivo ya existe")

El archivo ya existe


Comenzamos viendo los datos para limpiarlos...


In [57]:
raw_df = pd.read_csv("movies.csv")
raw_df.head()

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross
0,Blood Red Sky,(2021),"\nAction, Horror, Thriller",6.1,\nA woman with a mysterious illness is forced ...,\n Director:\nPeter Thorwarth\n| \n Star...,"21,062",121.0,NaN
1,Masters of the Universe: Revelation,(2021– ),"\nAnimation, Action, Adventure",5.0,\nThe war for Eternia begins again in what may...,"\n \n Stars:\nChris Wood, \nSara...","17,870",25.0,NaN
2,The Walking Dead,(2010–2022),"\nDrama, Horror, Thriller",8.2,\nSheriff Deputy Rick Grimes wakes up from a c...,"\n \n Stars:\nAndrew Lincoln, \n...","885,805",44.0,NaN
3,Rick and Morty,(2013– ),"\nAnimation, Adventure, Comedy",9.2,\nAn animated series that follows the exploits...,"\n \n Stars:\nJustin Roiland, \n...","414,849",23.0,NaN
4,Army of Thieves,(2021),"\nAction, Crime, Horror",NaN,"\nA prequel, set before the events of Army of ...",\n Director:\nMatthias Schweighöfer\n| \n ...,NaN,NaN,NaN


Como vemos con la funcion _info_, las columnas year, genre, rating, votes, runtime y gross tienen nulos.

In [58]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   MOVIES    9999 non-null   object 
 1   YEAR      9355 non-null   object 
 2   GENRE     9919 non-null   object 
 3   RATING    8179 non-null   float64
 4   ONE-LINE  9999 non-null   object 
 5   STARS     9999 non-null   object 
 6   VOTES     8179 non-null   object 
 7   RunTime   7041 non-null   float64
 8   Gross     460 non-null    object 
dtypes: float64(2), object(7)
memory usage: 703.2+ KB


#### b) Leer los datos del archivo

- **MOVIES:** indica el título de cada película o serie. 
- **YEAR:** indica el año de estreno de la película o el rango de años en los cuales fue emitido el programa. 
- **GENRE:** indica el/los género/s de la película o serie
- **RATING:** indica el puntaje de la pelicula en la escala del 1 al 10.
- **ONE-LINE:** se refiere a la sinopsis de la película o programa.
- **STARS:** actores que participan en el rodaje.
- **VOTES:** la cantidad de votos realizados en IMDB.
- **RunTime:** duración de la película.
- **Gross:** se refiere a las ganancias de la película o programa.

#### c) Realizar un análisis preliminar de valores nulos. Discutir qué acciones correctivas se creen pertinentes para resolver la presencia de estos valores.

- **MOVIES:** no tiene nulos. 
- **YEAR:** tiene nulos, lo que podría indicar que hay peliculas o series que no se estrenaron. También podría indicar que fue cancelada. Otras peliculas tienen números romanos en vez de un año de emisión o sólo palabras,lo que no tiene ninguna lógica. Una corrección sería reemplazar por NaN aquellos valores que no sean un año o un rango.
- **GENRE:** tiene nulos, y eso puede ser a causa de que no tiene una clasificacion clara.
- **RATING:** tiene nulos porque no se estrenó o es desconocida o todavía no fue evaluada.
- **ONE-LINE:** no tiene nulos, pero, hay películas que no tienen resumen, por lo cual, habría que reemplazar por NaN.
- **STARS:** no tiene valores nulos, pero si discernibles, porque podría no haber estrellas en la película o serie. En este caso, habría que reemplazar por NaN.
- **VOTES:** tiene nulos porque no se estrenó, es desconocida o todavía no fue evaluada.
- **RunTime:** tienen nulos porque podrían ser películas no estrenadas o series.
- **Gross:** tiene muchos nulos. Esto puede deberse a que no quieren mostrar la plata que perdieron o ganaron, no fue estrenada, o no hay registro de la recaudación.

Dependiendo del análisis que se quiera hacer de los datos, es la decision de borrar las filas con nulos o las columnas. 

#### Transformando los datos obtenemos:

In [59]:
preprocessed_df = raw_df.copy()
df_con_nulos = preprocessed_df[preprocessed_df["ONE-LINE"].str.contains("Plot")]
df_con_nulos["ONE-LINE"].value_counts()

ONE-LINE
\nAdd a Plot\n                                                                                                                                           1265
\nPlot under wraps.                                                                                                                                         7
\nPlot unknown.                                                                                                                                             7
\nPlot kept under wraps.                                                                                                                                    6
\nPlot under wraps. Adaptation of the Ubisoft game.                                                                                                         2
\nPlot unknown. Sequel to the 2019 film 'Knives Out'.                                                                                                       1
\nPlot speculated. Story pitched on Twitter

Como se puede ver, todos estos valores deberian ser reemplazados por NaN, ya que no aportan información sobre la sinopsis de la pelicula o serie.

Realizamos la siguiente transformación:

In [63]:
preprocessed_df['ONE-LINE'] = preprocessed_df['ONE-LINE'].replace(r'.*Plot.*', pd.NA, regex=True)
preprocessed_df


,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross
0,Blood Red Sky,(2021),"\nAction, Horror, Thriller",6.1,\nA woman with a mysterious illness is forced ...,\n Director:\nPeter Thorwarth\n| \n Star...,"21,062",121.0,NaN
1,Masters of the Universe: Revelation,(2021– ),"\nAnimation, Action, Adventure",5.0,\nThe war for Eternia begins again in what may...,"\n \n Stars:\nChris Wood, \nSara...","17,870",25.0,NaN
2,The Walking Dead,(2010–2022),"\nDrama, Horror, Thriller",8.2,\nSheriff Deputy Rick Grimes wakes up from a c...,"\n \n Stars:\nAndrew Lincoln, \n...","885,805",44.0,NaN
3,Rick and Morty,(2013– ),"\nAnimation, Adventure, Comedy",9.2,\nAn animated series that follows the exploits...,"\n \n Stars:\nJustin Roiland, \n...","414,849",23.0,NaN
4,Army of Thieves,(2021),"\nAction, Crime, Horror",NaN,"\nA prequel, set before the events of Army of ...",\n Director:\nMatthias Schweighöfer\n| \n ...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
9994,The Imperfects,(2021– ),"\nAdventure, Drama, Fantasy",NaN,<NA>,\n \n Stars:\nMorgan Taylor Camp...,NaN,NaN,NaN
9995,Arcane,(2021– ),"\nAnimation, Action, Adventure",NaN,<NA>,\n,NaN,NaN,NaN
9996,Heart of Invictus,(2022– ),"\nDocumentary, Sport",NaN,<NA>,\n Director:\nOrlando von Einsiedel\n| \n ...,NaN,NaN,NaN
9997,The Imperfects,(2021– ),"\nAdventure, Drama, Fantasy",NaN,<NA>,\n Director:\nJovanka Vuckovic\n| \n Sta...,NaN,NaN,NaN
